In [1]:
import os
import csv
from keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout,Activation
from keras.layers import Lambda
from keras import optimizers
from keras.layers.convolutional import MaxPooling2D,Convolution2D,Cropping2D

Using TensorFlow backend.


In [2]:
samples = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
samples=samples[1:]

In [3]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [4]:
import cv2
import numpy as np
import sklearn

In [5]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            for batch_sample in batch_samples:
                for i in range(3):
                    source_path=batch_sample[i]
                    filename=source_path.split('/')[-1]
                    #print(filename)
                    current_path='data/IMG/'+filename
                    image = cv2.imread(current_path)
                    images.append(image)
                measurement = float(line[3])
                correction=0.2
                measurements.append(measurement)
                measurements.append(measurement+correction)
                measurements.append(measurement-correction)

                image_flipped=[]
                measurement_flipped=[]
                for i in range(len(images)):
                    image_flipped.append(images[i])
                    measurement_flipped.append(measurements[i])

                images=images+image_flipped
                measurements=measurements+measurement_flipped

                del image_flipped
                del measurement_flipped

            X_train=np.array(images)
            y_train=np.array(measurements)
            print(X_train.shape)

            # trim image to only see section with road
            
            yield sklearn.utils.shuffle(X_train, y_train)

In [6]:
# batch_samples = samples[0:32]

# images = []
# angles = []
# for batch_sample in batch_samples:
#     # name = './IMG/'+batch_sample[0].split('/')[-1]
#     # center_image = cv2.imread(name)
#     # center_angle = float(batch_sample[3])
#     # images.append(center_image)
#     # angles.append(center_angle)
#     center = './data/'+batch_sample[0].split('/')[-1]
#     left = './data/'+batch_sample[1].split('/')[-1]
#     right = './data/'+batch_sample[2].split('/')[-1]
#     image_center = cv2.imread(center)
#     image_left = cv2.imread(left)
#     image_right = cv2.imread(right)
#     angle = float(batch_sample[6])
#     images.append([image_center, image_left, image_right])
#     #print(images)
#     angles.append(angle)
#     print(angles)


In [7]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [9]:
model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
# model.add(Lambda(lambda x: x/127.5 - 1.,
#         input_shape=(ch, row, col),
#         output_shape=(ch, row, col)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(34, 5, 5,activation='relu'))
print('Convolute')
# model.add(MaxPooling2D())
# model.add(Convolution2D(28, 5, 5,activation='relu'))
# model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(60))
model.add(Dropout(0.2))
# model.add(Dense(30))
# model.add(Dropout(0.3))
model.add(Dense(10))
#model.add(Dropout(0.2))
model.add(Dense(1))
print('Model Done')
#sgd = optimizers.SGD(lr=0.01)
model.compile(loss='mse',optimizer='adam')


Convolute
Model Done


In [12]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 156, 316, 34)  2584        lambda_1[0][0]                   
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 1676064)       0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 60)            100563900   flatten_1[0][0]                  
___________________________________________________________________________________________

In [ ]:
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator,nb_val_samples=len(validation_samples), nb_epoch=3)

Epoch 1/3
